<a href="https://colab.research.google.com/github/chrispoole70/langchain-tutorials/blob/main/agents/agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This tutorial is called [Build an Agent](https://python.langchain.com/docs/tutorials/agents/)

In [2]:
%pip install -qU "langchain[openai]"

In [ ]:
%pip install -qU langchain-tavily

In [ ]:
%pip install langgraph

In [39]:
import os
from pprint import pprint

from google.colab import userdata
from langchain.chat_models import init_chat_model
from langchain_tavily import TavilySearch
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

In [4]:
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

In [5]:
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [7]:
chat_model = init_chat_model(model='gpt-4o-mini', model_provider='openai')

## Define Tools

We need to define the tools that we want the agent to use. We are going to use a search engine called [Tavily](https://python.langchain.com/docs/integrations/tools/tavily_search/) that was built specifically for agents to use.

In [13]:
search = TavilySearch(max_results=2)

In [14]:
search_results = search.invoke('What is the weather in Stowe, Vermont?')

In [17]:
pprint(search_results)

{'answer': None,
 'follow_up_questions': None,
 'images': [],
 'query': 'What is the weather in Stowe, Vermont?',
 'response_time': 2.91,
 'results': [{'content': "{'location': {'name': 'Stowe', 'region': 'Vermont', "
                         "'country': 'United States of America', 'lat': "
                         "44.4653, 'lon': -72.685, 'tz_id': "
                         "'America/New_York', 'localtime_epoch': 1747681726, "
                         "'localtime': '2025-05-19 15:08'}, 'current': "
                         "{'last_updated_epoch': 1747681200, 'last_updated': "
                         "'2025-05-19 15:00', 'temp_c': 5.6, 'temp_f': 42.1, "
                         "'is_day': 1, 'condition': {'text': 'Light rain', "
                         "'icon': "
                         "'//cdn.weatherapi.com/weather/64x64/day/296.png', "
                         "'code': 1183}, 'wind_mph': 8.5, 'wind_kph': 13.7, "
                         "'wind_degree': 304, 'wind_dir': 'NW', 'pr

In [18]:
tools = [search]

In [19]:
chat_model_with_tools = chat_model.bind_tools(tools=tools)

In [20]:
response = chat_model_with_tools.invoke(input=[HumanMessage(content='What is the weather in Stowe, Vermont?')])

There's no generated response yet

In [25]:
response.content

''

But there is a tool call. However, this doesn't actually call the underlying function.

In [23]:
response.tool_calls

[{'name': 'tavily_search',
  'args': {'query': 'current weather Stowe Vermont', 'topic': 'general'},
  'id': 'call_wKRcCk62zpu7IDm41zekQDXL',
  'type': 'tool_call'}]

To have the tool call the function, you can create a LangGraph agent.

## Create the agent

You can use the chat model that isn't bound to the tools because that will happen when the agent gets created

In [29]:
agent_executor = create_react_agent(model=chat_model, tools=tools)

## Run the agent

Now when the agent is run, it will take action by calling the tool's underlying function

In [30]:
response = agent_executor.invoke(
    input={'messages': [HumanMessage(content='What is the weather in Stowe, Vermont?')]}
)

In [35]:
[message.pretty_print() for message in response['messages']]

================================ Human Message =================================

What is the weather in Stowe, Vermont?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_SbPBGY7koOCDWdUomE31GQr7)
 Call ID: call_SbPBGY7koOCDWdUomE31GQr7
  Args:
    query: current weather in Stowe Vermont
    topic: general
================================= Tool Message =================================
Name: tavily_search

{"query": "current weather in Stowe Vermont", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in Stowe, Vermont", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Stowe', 'region': 'Vermont', 'country': 'United States of America', 'lat': 44.4653, 'lon': -72.685, 'tz_id': 'America/New_York', 'localtime_epoch': 1747684593, 'localtime': '2025-05-19 15:56'}, 'current': {'last_updated_epoch': 1747683900, 'last_updated': '2025-05-19 15:45', 'temp_c': 8.9, 'temp

[None, None, None, None]

## Streaming Messages

Streaming each response message is a better UX especially if it can take awhile for the agent to run. Notice that the `stream_mode` is "values" which means emit the value after each step in the agent graph.

In [36]:
for step in agent_executor.stream(
    input={'messages': [HumanMessage(content='What is the weather in Stowe, Vermont?')]},
    stream_mode='values'
):
  step['messages'][-1].pretty_print()

================================ Human Message =================================

What is the weather in Stowe, Vermont?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_BYqiSFu6FPxqB6AqsGx3SFEg)
 Call ID: call_BYqiSFu6FPxqB6AqsGx3SFEg
  Args:
    query: current weather in Stowe, Vermont
    topic: general
================================= Tool Message =================================
Name: tavily_search

{"query": "current weather in Stowe, Vermont", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in Stowe, Vermont", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Stowe', 'region': 'Vermont', 'country': 'United States of America', 'lat': 44.4653, 'lon': -72.685, 'tz_id': 'America/New_York', 'localtime_epoch': 1747685334, 'localtime': '2025-05-19 16:08'}, 'current': {'last_updated_epoch': 1747684800, 'last_updated': '2025-05-19 16:00', 'temp_c': 8.9, 'te

Or you can emit each message token-by-token

In [38]:
for step, metadata in agent_executor.stream(
    input={'messages': [HumanMessage(content='What is the weather in Stowe, Vermont?')]},
    stream_mode='messages'
):
  if metadata["langgraph_node"] == "agent" and (text := step.text()):
      print(text, end="|")

The| current| weather| in| St|owe|,| Vermont| is| as| follows|:

|-| **|Temperature|**|:| |8|.|9|°C| (|48|°F|)
|-| **|Condition|**|:| Over|cast|
|-| **|Wind| Speed|**|:| |7|.|6| mph| (|12|.|2| k|ph|)| from| the| northwest|
|-| **|Humidity|**|:| |77|%
|-| **|Visibility|**|:| |16| km| (|9| miles|)
|-| **|Feels| Like|**|:| |6|.|9|°C| (|44|.|5|°F|)

|You| can| find| more| information| on| the| weather| [|here|](|https|://|www|.weather|api|.com|/|).|

## Adding in Memory

By default, a LangGraph agent is stateless. To add state to it, you have to add a checkpointer so that the conversation history can be preserved.

In [40]:
memory = MemorySaver()

In [41]:
agent_executor = create_react_agent(model=chat_model, tools=tools, checkpointer=memory)

Don't forget to pass a config to the model that specifies which conversation to use

In [42]:
config = {"configurable": {"thread_id": "abc123"}}

In [43]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hi Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 771, 'total_tokens': 782, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_129a36352a', 'id': 'chatcmpl-BZ1OwdEk3UYG3b8CLihuxEv2QjvJf', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--156d071c-dffc-472c-b7d6-da2efdf35ce5-0', usage_metadata={'input_tokens': 771, 'output_tokens': 11, 'total_tokens': 782, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----


In [44]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 794, 'total_tokens': 800, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BZ1PATYaHK4TwAW4NO5cxS5dhQs6i', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--93d459b8-f78b-4cb5-a78d-049607c7ba52-0', usage_metadata={'input_tokens': 794, 'output_tokens': 6, 'total_tokens': 800, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----
